In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from datetime import date, timedelta

In [2]:
spark = SparkSession \
    .builder \
    .appName("klaus_session") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/18 23:48:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
df = spark.read.parquet("/opt/airflow/silver_layer/")

In [21]:
df.show(1,vertical=True)

-RECORD 0-----------------------------------
 id_brewery          | 6d14b220-8926-452... 
 name_brewery        | 10 Barrel Brewing Co 
 brewery_type        | large                
 phone_brewery       | 5415851007           
 name_country        | United States        
 name_state_province | Oregon               
 name_city           | Bend                 
 postal_code_brewery | 97701-9847           
 latitude_brewery    | 44.08683531          
 longitude_brewery   | -121.281706          
 name_address_1      | 62970 18th St        
 name_address_2      | NULL                 
 name_address_3      | NULL                 
 updated_at          | 2024-10-18 23:46:49  
 name_state          | Oregon               
only showing top 1 row



In [22]:
df = df.withColumn('index',row_number().over(Window.partitionBy(col("id_brewery")).orderBy(col("updated_at").desc())))
df = df.filter(df.index == 1) 
df = df.drop("index")

In [29]:
df_gold = df.groupBy(df.name_state, df.brewery_type).count()
df_gold = df_gold.withColumnRenamed("count", "breweries_quantity")

In [30]:
df_gold.orderBy('count', ascending=False).show()


+-------------+------------+------------------+
|   name_state|brewery_type|breweries_quantity|
+-------------+------------+------------------+
|       Oregon|       large|                 4|
|    Minnesota|       micro|                 3|
|      Indiana|       micro|                 3|
|      Arizona|       micro|                 3|
|   California|       micro|                 2|
|        Texas|       micro|                 2|
|       Oregon|     brewpub|                 2|
|    Wisconsin|       micro|                 2|
|     New York|     brewpub|                 2|
|         Iowa|       micro|                 1|
|     Virginia|       micro|                 1|
|   California|       large|                 1|
|     Colorado|     brewpub|                 1|
|       Nevada|       micro|                 1|
|     Michigan|       micro|                 1|
|      Vermont|       micro|                 1|
|     Delaware|       micro|                 1|
|        Laois|       micro|            

In [32]:
#Writing DataFrame
df_gold.write \
.format("parquet") \
.mode("append") \
.option("mergeSchema", "true") \
.save(f'/opt/airflow/gold_layer/')